# This notebook illustrates a 'fast' method of regridding data where the point-to-point mappings remain the same for many datasets

In [7]:
%config Completer.use_jedi = False
import pickle
import numpy as np
from scipy.interpolate import griddata
import pyproj as proj
import cartopy.crs as ccrs
import cartopy
import mask
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from tools import lonlat_to_xy
from scipy.interpolate import interp2d
import metpy.calc as mpcalc
from metpy.units import units
from cartoplot import cartoplot
import tqdm
import matplotlib.animation as animation
from IPython import display
from scipy.interpolate import griddata, LinearNDInterpolator
from regrid import regrid
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator

In [2]:
ease_lons, ease_lats = mask.get('lon'), mask.get('lat')

my_mask = mask.get('mask')

ease_dx, ease_dy = mpcalc.lat_lon_grid_deltas(ease_lons, ease_lats)

ease_dx = abs(ease_dx); ease_dy = abs(ease_dy)

ease_cos_lons, ease_sin_lons = np.cos(np.deg2rad(ease_lons)), np.sin(np.deg2rad(ease_lons))

In [3]:
d = Dataset('/media/robbie/TOSHIBA EXT/E5/2021.nc')
ERA_lons = np.array(d['longitude'])
ERA_lats = np.array(d['latitude'])

ERA5_lon_grid = np.array([np.array(ERA_lons), ] * 121)

ERA5_lat_grid = np.array([np.array(ERA_lats), ] * 1440).T

ERA_x_grid, ERA_y_grid = lonlat_to_xy(ERA5_lon_grid, ERA5_lat_grid,hemisphere='n')

dx, dy = mpcalc.lat_lon_grid_deltas(ERA_lons, ERA_lats)

cos_lons, sin_lons = np.cos(np.deg2rad(ERA5_lon_grid)), np.sin(np.deg2rad(ERA5_lon_grid))

In [4]:
args = proj.Proj(proj="aeqd", lat_0=90, lon_0=0, datum="WGS84", units="m")

crs_wgs = proj.Proj(init='epsg:4326')  # assuming you're using WGS84 geographic

xout, yout = proj.transform(crs_wgs, args, np.array(ease_lons),np.array(ease_lats))

xin, yin = proj.transform(crs_wgs, args, np.array(ERA5_lon_grid),np.array(ERA5_lat_grid))

/home/robbie/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/robbie/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/robbie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-

In [ ]:
points = np.column_stack((ERA_x_grid.ravel(),ERA_y_grid.ravel()))

tri = Delaunay(points)  # Compute the triangulation

In [5]:
big_array = np.zeros((d['v10'].shape[0],361,361))

for ind in tqdm.trange(d['v10'].shape[0]):

    v10 = np.array(d['v10'][ind])
    u10 = np.array(d['u10'][ind])

    interpolator = LinearNDInterpolator(tri, u10.ravel())
    ease_u10 = interpolator((xout,yout))

    interpolator = LinearNDInterpolator(tri, v10.ravel())
    ease_v10 = interpolator((xout,yout))
    
    x10 = np.multiply(ease_u10,ease_cos_lons) - np.multiply(ease_v10,ease_sin_lons)
    y10 = np.multiply(ease_u10,ease_sin_lons) + np.multiply(ease_v10,ease_cos_lons)
    
    x10 = x10* units.meter / units.second
    y10 = y10* units.meter / units.second

    vort = mpcalc.vorticity(x10, y10, dx=ease_dx,dy= ease_dy)
    
    big_array[ind] = vort

100%|██████████| 472/472 [00:35<00:00, 13.37it/s]


In [6]:
fig = plt.figure(figsize=(10,10))

ax = plt.axes(projection=ccrs.NorthPolarStereo())

ax.set_extent([-180, 180, 90, 65], ccrs.PlateCarree())    

ax.add_feature(cartopy.feature.LAND, edgecolor='black',zorder=1)

ax.margins()

bg = ax.pcolormesh(ease_lons, 
                   ease_lats, 
                   big_array[0][:-1,:-1]*1e5, 
                    vmin = -5, 
                    vmax = 5,
                    transform=ccrs.PlateCarree(),
                    cmap='PRGn',
                    alpha=1)

fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)

########################################################

def animate(frame):
    
    if frame % 10 == 0: #Track progress
        print(frame)

    a_to_set = big_array[frame][:-1,:-1].ravel() * 1e5
    bg.set_array(a_to_set)


#######################################################

ani = animation.FuncAnimation(fig,
                              animate,
                             frames= range(0,5),                              
#                              frames= range(0,big_array.shape[0]),
                             )    

video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

0
0
